# Download Dataset

In [1]:
using NPZ
using Flux
train_ds_x = permutedims(npzread("../dataset/TF/train_x.npy"),[2,3,4,1])
# train_ds_orig_x = permutedims(npzread("../dataset/TF/train_x.npy"),[2,3,4,1])
train_ds_y = hcat([Flux.onehot(idx,0:7) for idx=npzread("../dataset/TF/train_y.npy")]...)
val_ds_x = permutedims(npzread("../dataset/TF/val_x.npy"),[2,3,4,1])
val_ds_y = hcat([Flux.onehot(idx,0:7) for idx=npzread("../dataset/TF/val_y.npy")]...)


8×800 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  1  ⋅  ⋅     1  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  …  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 1  ⋅  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [2]:
using Statistics
# Compute mean 
ds_mean =  mean(train_ds_x)
ds_sqrt_var = sqrt(var(train_ds_x))

0.758846f0

In [3]:
# Use the DataLoader
trainingData = Flux.DataLoader((train_ds_x, train_ds_y), batchsize=64,shuffle=true)
valData = Flux.DataLoader((val_ds_x, val_ds_y), batchsize=64)

MLUtils.DataLoader{Tuple{Array{Float32, 4}, Flux.OneHotArray{UInt32, 8, 1, 2, Vector{UInt32}}}, Random._GLOBAL_RNG, Val{nothing}}(([5.0665418f-5 0.000640263 … 0.00015216086 0.00020779307; 0.00065564585 0.00055528636 … 4.3868815f-5 5.2983873f-5; … ; 0.00045395162 0.0009790196 … 3.52279f-5 1.991613f-5; 0.00039587912 0.0006911695 … 0.0001233202 0.00014793497;;;; 5.6987665f-5 0.0012965294 … 6.1849714f-5 4.4529566f-5; 0.00048752356 0.0007681608 … 6.299725f-5 6.3923886f-5; … ; 0.01048574 0.0048514893 … 7.5378244f-5 8.461485f-5; 0.003013047 0.01135004 … 3.9271235f-5 3.2113283f-5;;;; 0.0018243344 0.008706955 … 0.00010042758 3.7692138f-5; 0.0012148265 0.013375979 … 6.976531f-5 2.0443462f-5; … ; 0.0034393203 0.028048309 … 2.7227365f-5 2.537202f-5; 0.0064670327 0.02280002 … 0.00011638377 3.0844007f-5;;;; … ;;;; 0.0021131248 0.0072379573 … 7.941158f-5 2.0045554f-5; 0.0009966363 0.005381674 … 9.4765826f-5 2.001028f-5; … ; 0.00026640831 0.03726436 … 0.00013899719 0.00011997786; 0.016938007 0.0304880

In [4]:
# Model Definition

model = Chain(
    Upsample(:bilinear,size=(32,32)),
    x->(x .- ds_mean) ./ ds_sqrt_var,
    Conv((3,3),1=>32, Flux.relu),
    Conv((3,3),32=>64, Flux.relu),
    MaxPool((2,2)),
    Dropout(0.25),
    Flux.flatten,
    Dense(12544=>128,relu),
    Dropout(0.5),
    Dense(128=>8),
)
# Checking 
model(train_ds_x[:,:,:,1:1])

8×1 Matrix{Float32}:
  0.2695503
 -0.3293139
 -0.21712938
  0.27615845
 -0.2459599
 -0.4050064
 -0.06462313
 -0.25922725

In [5]:
# Loss Function
function loss(x,y)
    ŷ = model(x)
    Flux.Losses.logitcrossentropy(ŷ,y)
end

# Check Accuracy
function getAccuracy(m,d)
    acc = 0
    for (x,y)=d
        ŷ = model(x)
        acc += sum(Flux.onecold(ŷ) .== Flux.onecold(y)) / size(x)[end]
    end
    acc/length(d)
end

# Total Loss
function loss_tot(d)
    l = 0
    for (x,y) = d
        l+= loss(x,y)
    end
    return l/length(d)
end

loss_tot (generic function with 1 method)

In [6]:
# Goes very slow :''(
#@show loss_tot(valData) 
testmode!(model,true)
@show getAccuracy(model,valData)

getAccuracy(model, valData) = 0.15144230769230768


0.15144230769230768

In [7]:
# Does not Learn
opt = Flux.Optimise.ADAM()
ps = Flux.params(model)
testmode!(model,false)

evalcb() = println("Loss: $(loss_tot(valData))")

Flux.@epochs 10 Flux.train!(
    loss,
    ps,
    trainingData,
    opt,
    cb=Flux.throttle(evalcb,10),
)

testmode!(model,true)
@show getAccuracy(model,valData)


┌ Info: Epoch 1
└ @ Main C:\Users\detter55\.julia\packages\Flux\js6mP\src\optimise\train.jl:154


Loss: 2.1415434
Loss: 1.7251213
Loss: 1.5278786


┌ Info: Epoch 2
└ @ Main C:\Users\detter55\.julia\packages\Flux\js6mP\src\optimise\train.jl:154


Loss: 1.409769
Loss: 1.2335472
Loss: 1.1429584


┌ Info: Epoch 3
└ @ Main C:\Users\detter55\.julia\packages\Flux\js6mP\src\optimise\train.jl:154


...

┌ Info: Epoch 9
└ @ Main C:\Users\detter55\.julia\packages\Flux\js6mP\src\optimise\train.jl:154


Loss: 0.7343285
Loss: 0.7963575
Loss: 0.7309685


┌ Info: Epoch 10
└ @ Main C:\Users\detter55\.julia\packages\Flux\js6mP\src\optimise\train.jl:154


Loss: 0.76419026
Loss: 0.73187727
Loss: 0.63137
getAccuracy(model, valData) = 0.8473557692307693


0.8473557692307693